In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


In [2]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
# 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
# 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
# 별점을 시청횟수로 해석해서 생각하겠습니다.
# 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'counts']
data = ratings[using_cols]
data.head(10)

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [9]:
# 유저 수
data['user_id'].nunique()

6039

In [10]:
# 영화 수
data['movie_id'].nunique()

3628

In [11]:
# 인기 많은 영화 30개
movie_count = data.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [12]:
# 유저별 몇 개의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [13]:
# 유저별 count횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

In [14]:
# 내가 좋아하는 영화 5개 
my_favorite = [110 , 480 ,1198 ,589 ,593]

# 'sh'이라는 user_id가 위 영화를  5회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['sh']*5, 'movie_id': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':['sh']})['user_id'].any():  # user_id에 'sh'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,sh,110,5
1,sh,480,5
2,sh,1198,5
3,sh,589,5
4,sh,593,5


In [15]:
# 고유한 유저를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()
# 유저 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [16]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['sh'])    # 총 6039명의 유저 에서 1명추가 6040명(idx=6039)
print(movie_to_idx[589])

6039
92


In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')
    
# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,87,5
1,6039,107,5
2,6039,120,5
3,6039,92,5


## CSR Matrix 만들기

In [18]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [19]:
# MF implicit

from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

In [23]:
#내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악
sh, mtd110 = user_to_idx['sh'], movie_to_idx[110]
sh_vector, mtd110_vector = als_model.user_factors[sh], als_model.item_factors[mtd110]


In [24]:
sh_vector

array([-0.3617932 , -0.2083037 ,  0.01389139,  0.02030345,  0.00612969,
        0.19602716,  0.06652939,  0.3922777 ,  0.0683066 ,  0.1287643 ,
       -0.38635072, -0.11391551, -0.5207075 ,  0.51922417,  0.72709   ,
       -0.20076653,  0.39977303, -0.04317514,  0.08925647,  0.63062274,
        0.00266833, -0.03117319,  0.21173939,  0.29014271, -0.6265656 ,
       -0.38907087,  0.05223315, -0.03054998, -0.53395987, -0.25739247,
       -0.8355721 ,  0.24840339,  0.24288315,  0.22709818, -0.07568221,
       -0.0929366 , -0.19144598, -0.3370674 , -0.11076754,  0.43127567,
       -0.12397194, -0.2937209 ,  0.5587928 ,  0.19350791, -0.1024675 ,
        0.7158418 , -0.25171065, -0.32110602, -0.45023674, -0.35396302,
       -0.04799958, -0.49991477,  0.20084625, -0.40296775, -0.02264782,
       -0.587643  ,  0.38571763,  0.14413317,  0.5556522 , -0.16862588,
        0.03050453,  0.83219516,  0.05225057, -0.19329163, -0.32950166,
       -0.00809423, -0.15623376,  0.64193743, -0.2466775 ,  0.01

In [25]:
mtd110_vector

array([ 4.19057580e-03, -2.54552253e-03,  3.39837782e-02,  4.41158051e-03,
       -1.98662151e-02,  1.86440218e-02, -3.64695415e-02, -3.48966313e-03,
        2.71358173e-02,  2.87220757e-02, -2.11295374e-02, -1.57022150e-03,
       -2.37753969e-02,  3.68825272e-02,  4.12334921e-03,  2.70908438e-02,
        3.98610197e-02,  1.12190945e-02, -3.33795957e-02,  4.58759815e-03,
       -3.71992067e-02,  1.25038589e-03,  4.05763015e-02,  3.72947752e-02,
       -3.55978571e-02, -1.46074884e-03,  3.18185650e-02,  1.44260451e-02,
        2.46468000e-02, -4.91959341e-02, -1.25429174e-02,  2.21939068e-02,
        3.78217623e-02,  3.67286354e-02,  1.06735975e-02, -1.01194317e-02,
       -1.92492418e-02,  6.38962258e-04,  4.44613351e-03,  2.78394520e-02,
        5.83694642e-03, -1.80506073e-02, -8.96667130e-03, -3.19419019e-02,
        1.71901099e-02,  5.88544160e-02,  2.42160610e-03, -8.50724801e-03,
       -7.86651112e-03, -1.26533108e-02,  3.68799344e-02, -3.83553770e-03,
        3.92952003e-02, -

In [26]:
np.dot(sh_vector, mtd110_vector) ## 선호도 0.87 나옴

0.87560356

In [27]:
# 그 외의 영화와 선호도 비교
mtd2829 = movie_to_idx[2829]
mtd2829_vector = als_model.item_factors[mtd2829]

In [28]:
mtd2829_vector

array([ 3.86330037e-04,  1.15673458e-02,  1.11260265e-02,  1.68767394e-04,
       -7.06709269e-03, -7.50306249e-03,  2.31718412e-03,  3.96579364e-03,
        5.06834034e-03,  8.38883105e-04,  1.04220323e-02, -3.44946911e-03,
        7.21291639e-04, -6.32155174e-03,  3.34122963e-03,  3.30412225e-03,
        6.98053685e-04,  6.39558537e-03,  1.05703892e-02,  1.77831240e-02,
        1.55422911e-02,  8.77579022e-03,  1.15240449e-02,  1.58005767e-02,
        1.21262753e-02,  1.40721165e-02,  7.67666567e-03,  1.36241512e-02,
        3.69221647e-03,  1.76308230e-02, -8.56465602e-04,  1.71028581e-02,
       -2.07356573e-03, -4.82731353e-04,  1.39479684e-02,  1.98539835e-03,
        4.21130238e-03,  1.16123324e-02,  1.12517104e-02, -1.35897659e-03,
        8.40455573e-03, -8.56093224e-03,  8.06366745e-03,  1.28436089e-02,
        2.29620538e-03,  8.23424850e-03, -4.55710018e-04,  8.90644081e-03,
        1.15194842e-02,  7.11088907e-03,  5.43768983e-03,  1.16472114e-02,
        4.50858707e-03,  

In [29]:
np.dot(sh_vector, mtd2829_vector) ## 그 외의 영화에 대해선 0.03

0.037150756

## 비슷한 영화 추천 받기

In [30]:
# 비슷한 아티스트 찾기

favorite_movie = 110
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(87, 1.0),
 (48, 0.32488894),
 (92, 0.30641532),
 (1474, 0.26252773),
 (107, 0.26199853),
 (3489, 0.260618),
 (2036, 0.25719208),
 (2695, 0.2531629),
 (3026, 0.25071847),
 (3181, 0.25016615),
 (3154, 0.24580584),
 (2408, 0.24559827),
 (3085, 0.24376337),
 (3139, 0.24243562),
 (3168, 0.24199456)]

In [31]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]


[110,
 2028,
 589,
 144,
 480,
 130,
 298,
 2785,
 1902,
 3570,
 2509,
 1896,
 3775,
 2760,
 554]

In [32]:
movies['title']

0                         Toy Story (1995)
1                           Jumanji (1995)
2                  Grumpier Old Men (1995)
3                 Waiting to Exhale (1995)
4       Father of the Bride Part II (1995)
                       ...                
3878               Meet the Parents (2000)
3879            Requiem for a Dream (2000)
3880                      Tigerland (2000)
3881               Two Family House (2000)
3882                 Contender, The (2000)
Name: title, Length: 3883, dtype: object

In [33]:
# 영화넘버에 따른영화 이름 가져오기
fM = [idx_to_movie[i[0]] for i in similar_movie]
[movies['title'][i-1] for i in fM] 

['Taxi Driver (1976)',
 'Sleeping Beauty (1959)',
 'Batman (1989)',
 'Bad Boys (1995)',
 'King of the Hill (1993)',
 'Frankie Starlight (1995)',
 'Quiz Show (1994)',
 'Communion (a.k.a. Alice, Sweet Alice/Holy Terror) (1977)',
 'Nightmare on Elm Street 3: Dream Warriors, A (1987)',
 'Moonraker (1979)',
 'Metroland (1997)',
 'Klute (1971)',
 'Steel Magnolias (1989)',
 'Dudley Do-Right (1999)',
 'Mamma Roma (1962)']

In [34]:
# 내가 가장 좋아할만한 영화 추천받기
user = user_to_idx['sh']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(175, 0.43855786),
 (172, 0.3688791),
 (124, 0.3041414),
 (157, 0.29703313),
 (48, 0.29333323),
 (200, 0.28736162),
 (141, 0.24554995),
 (44, 0.22630613),
 (51, 0.21832165),
 (62, 0.21395773),
 (27, 0.21282521),
 (99, 0.19671527),
 (22, 0.18997306),
 (117, 0.17084904),
 (75, 0.1546008),
 (468, 0.15411608),
 (82, 0.15162852),
 (194, 0.15107612),
 (197, 0.15077192),
 (276, 0.13898298)]

In [35]:
[idx_to_movie[i[0]] for i in movie_recommended]

[1580,
 1291,
 2571,
 318,
 2028,
 1240,
 457,
 260,
 608,
 2916,
 1721,
 2858,
 1270,
 1196,
 1610,
 3160,
 1544,
 1036,
 1387,
 908]

In [36]:
rM = [idx_to_movie[i[0]] for i in movie_recommended]
[movies['title'][i-1] for i in rM] 

['Kicked in the Head (1997)',
 'Hype! (1996)',
 'Mommie Dearest (1981)',
 'Suture (1993)',
 'Sleeping Beauty (1959)',
 'Stand by Me (1986)',
 'Getting Even with Dad (1994)',
 'Little Princess, A (1995)',
 'Hellraiser: Bloodline (1996)',
 'On Any Sunday (1971)',
 'Wedding Singer, The (1998)',
 'Hairspray (1988)',
 'Koyaanisqatsi (1983)',
 'GoodFellas (1990)',
 'Phantoms (1998)',
 'Wirey Spindell (1999)',
 'Contact (1997)',
 'Ghost and the Darkness, The (1996)',
 'Michael (1996)',
 'Wizard of Oz, The (1939)']

In [37]:
# 왜 Little Princess를 추천 추천에 기여한 정도 표시
Little_Princess = movie_to_idx[260]
explain = als_model.explain(user, csr_data, itemid=Little_Princess)

In [38]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(1198, 0.21223249192223592),
 (480, 0.044921441377455726),
 (110, 0.007007826675071542),
 (589, -0.007519993178300945),
 (593, -0.03137383255787537)]

In [39]:
rtM =[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

In [40]:
# 인덱스 번호에 맞는 영화 이름으로 변환
rtM_E = [idx_to_movie[i[0]] for i in rtM]
[movies['title'][i-1] for i in rtM_E] 

['Saludos Amigos (1943)',
 'Glory (1989)',
 'Big Sleep, The (1946)',
 'Foreign Correspondent (1940)',
 'Robin Hood: Prince of Thieves (1991)']

## 회고

- 사용자와 아이템 개수를 바탕으로 정확한 사이즈의 CSR matrix를 정상적으로 만들어 보았다.
- MF 모델이 그럴듯한 추천이 이루어 졌다.
- MF 모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정해보았다.
- 전체 data 행렬을 CSR Matrix로 분해하여 학습 연산 리소스도 줄이고 효과적으로 데이터를 분석할 수 있게 만든 점이 인상 깊었다.
- 필요한 벡터값만을 가지고 CSR Matrix내의 값을 바로 내적으로 연산할수 있는것이 굉장히 효과적이였다.
- 모델이 자체적으로 영화 벡터간 기여도 유사도를 측정하여 각각의 유사도가 잘 분석 된 것을 확인 할 수 있었다.
- 인덱스를 영화넘버로 변환하는 과정에서 전체 영화넘버(3883)를 초과하는 영화넘버가 나오는 경우가 있었다. 추후 코드확인